# Lecture, visualisation et traitements de radargramme

(développé à l'Université Paris Saclay par Albane Saintenoy avec l'aide d'Emmanuel Léger, Simon Védrine, Jean-Michel Friedt, Samuel Lelièvre et Nicolas Thiéry)

<img src=./20230322_095856.jpg width="400" height="800" />

Cette première cellule de code jupyter permet d'importer les librairies et les fonctions prédéfinies nécessaires à la lecture, visualisation et traitement de radargramme. Les fonctions sont dans les fichiers utils.py et processing_tools.py placés dans le dossier où se trouve cette feuille de calculs.

In [7]:
from utils import *
from processing_tools import *
from ipywidgets import interact, IntSlider, FloatSlider, FloatLogSlider
import pandas as pd
import os.path
import numpy as np
import readgssi.functions
from readgssi import readgssi
from mpl_toolkits.axes_grid1 import make_axes_locatable
import warnings
warnings.filterwarnings('ignore')
warnings.warn("WARNING")
plt.rcParams.update({'font.size': 22})

La fonction ci-après permet de tenir compte de la numérotation particulière de chaque fichier de données, lorsqu'il y en a plus que 10. Elle pourrait être mise dans le fichier utils.py mais elle est pratique à avoir sous la main si besoin de s'adapter au cas particulier des radargrammes à traiter.

In [2]:
def lecture(path, ip):
    '''fonction de lecture des fichiers de données'''
    num_profil = ip
    if num_profil < 10 :
        filename = path + "FILE__00" + str(num_profil) + ".DZT"
    else :
        filename = path + "FILE__0" + str(num_profil) + ".DZT"
    hdr, arrs, gps = readgssi.readgssi(infile = filename, verbose=None, zero=[0])
    data = arrs[0]
    return data

## Lecture d'un fichier de données

Schéma des acquisitions :
<img src=./notes_terrainG2.jpg width="400" height="800" />

À partir de ce schéma, on peut mettre à jour la longueur des profils et lire le premier profil du dossier de données. Ne pas faire attention aux messages WARNING de la commande de lecture readgssi. Les format de données que l'on peut utiliser son ASCII, HDF5 (SORTIES GPRMAX), DZT (GSSI) et RD3 (RAMAC MALA). 

In [9]:
a=3+2
a

5

In [3]:
longueur = 30 #longueur du profil en m
nombre_de_profils = 57
#Lecture du fichier de données
path = "/home/sainteno/Donnees/2023/Hautefeuille/RadarG2/"
filename = path + "FILE__001.DZT"

hdr, arrs, gps = readgssi.readgssi(infile = filename, antfreq=None, verbose=False, zero=[0])
dt = hdr['ns_per_zsample']*10**9
rh_ant = hdr['rh_ant']
timezero_acq = hdr['rhf_position']

data = arrs[0]
ns, ntr = data.shape
dx = longueur/ntr
dx_dt = dx, dt 

2024-02-11 19:48:18 - WARNING: no DZG file found for GPS input
--------------------WARNING - PLEASE READ---------------------
2024-02-11 19:48:18 - WARNING: could not read frequency for antenna name "'HS350US'"
2024-02-11 19:48:18 - WARNING: trying to use frequencies of None MHz (estimated)...
2024-02-11 19:48:18 - more info: rh_ant=['HS350US', None, None, None]
2024-02-11 19:48:18 -            known_ant=[False, None, None, None]
2024-02-11 19:48:18 - please submit a bug report with this warning, the antenna name and frequency
2024-02-11 19:48:18 - at https://github.com/iannesbitt/readgssi/issues/new
2024-02-11 19:48:18 - or send via email to ian (dot) nesbitt (at) gmail (dot) com.
2024-02-11 19:48:18 - if possible, please attach a ZIP file with the offending DZT inside.
--------------------------------------------------------------


Affichons les différents paramètres d'acquisition du premier radargramme

In [4]:
print('Nombre de traces =', ntr)
print('Pas spatial =', dx, 'm')
print('Pas temporel =', dt, 'ns')
print("Antenne utilisée : ", rh_ant[0])
print('timezero_acquisition =', timezero_acq, 'ns')

Nombre de traces = 1779
Pas spatial = 0.016863406408094434 m
Pas temporel = 0.15638075430393122 ns
Antenne utilisée :  HS350US
timezero_acquisition = -8.88888931274414 ns


## Visualisation du radargramme brut

Affichons le radargramme brut, sans aucun traitement. Vous pouvez changer la valeur clip, qui joue sur l'échelle de couleur. Quand clip=0.3, la barre de couleur est entre -0.3 amplitude max et +0.3 amplitude max. Plus vous diminuer cette valeur, plus les grandes amplitudes sont saturées en couleur, mais plus vous faites apparaître les détails plus tardifs.

In [5]:
clip_default = 0.3
@interact
def f(clip=FloatLogSlider(value=clip_default, min=-5, max=0, step=.5)):
    plot_radargram(data, dx_dt, clip)

interactive(children=(FloatLogSlider(value=0.3, description='clip', max=0.0, min=-5.0, step=0.5), Output()), _…

Regardons maintenant les traces qui composent ce radargramme. Vous pouvez changer le numéro n de la trace à afficher. L'amplitude du champ électrique enregistrée au cours du temps est marquée par une onde directe de forte amplitude en début de trace suivi d'une décroissance plus ou moins rapide au cours du temps. 

In [6]:
n_default = 10
@interact
def f(n=IntSlider(value=n_default, min=1, max=ntr, step=1)):
    fig, ax = plt.subplots(figsize=(5, 10))
    ax.plot(data[:, n-1], np.arange(ns)*dt)
    plt.ylabel('Temps (ns)')
    plt.xlabel('Amplitude')
    plt.ylim(ns*dt, 0)
    plt.minorticks_on()
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    return


interactive(children=(IntSlider(value=10, description='n', max=1779, min=1), Output()), _dom_classes=('widget-…

Regardons ce qui se passe lorsque l'on tronque les traces de tout le signal qui arrive avant la valeur initiale timezero_acq qui est ajusté sur le temps d'arrivée du premier minimum de l'amplitude de l'onde directe. On peut décider de prendre un autre temps zéros comme temps de référence sur la trace. 

In [9]:
# Choix du temps t0
t0_default =  np.abs(timezero_acq) # en ns
clip = 0.3
@interact
def f(t0 = FloatSlider(value=t0_default, min=0.0, max=20, step=dt)):
    global data_t0
    data_t0 = time_zero(data, dx_dt, t0)
    plot_radargram(data_t0, dx_dt, clip)

interactive(children=(FloatSlider(value=8.88888931274414, description='t0', max=20.0, step=0.15638075430393122…

In [10]:
# Sauvons la valeur choisie:
t0 = np.abs(timezero_acq)

## Atténuation de l'onde directe

Plusieurs traitements existent pour "retirer" l'onde directe pour pouvoir mettre en évidence d'éventuelles réflexions "cachées" dessous. Nous allons regarder le retrait de la trace moyenne, puis le filtrage par valeurs singulières (SVD filter).

In [11]:
# Retrait de la trace moyenne
data_t0m = mean_tr_rm(data_t0)
clip_default = 0.3
@interact
def f(clip=FloatLogSlider(value=clip_default, min=-5, max=0, step=.5)):
    plot_radargram(data_t0m, dx_dt, clip)

interactive(children=(FloatLogSlider(value=0.3, description='clip', max=0.0, min=-5.0, step=0.5), Output()), _…

In [13]:
# Filtrage par SVD
SVmin_default = 1
SVmax_default = 200
clip_default=0.3
ns, ntr = data_t0.shape
@interact
def f(SVmin=IntSlider(value=SVmin_default, min=0, max=ns-1, step=1), 
      SVmax=IntSlider(value=SVmax_default, min=1, max=ns, step=1),
      clip=FloatLogSlider(value=clip_default, min=-5, max=0, step=.5)):
    global dataSVD
    dataSVD = filt_SVD(data_t0, ns, ntr, SVmin, SVmax)
    plot_radargram(dataSVD, dx_dt, clip)

interactive(children=(IntSlider(value=1, description='SVmin', max=455), IntSlider(value=200, description='SVma…

In [14]:
#Sauvons les limites du filtrage SVD
SVmin = 2
SVmax = 200

## Application d'un gain

Même après retrait de l'onde directe, il est toujours difficile de voir les réflexions tardives sans saturer les amplitudes des réflexions qui arrivent tôt. Pour améliorer cela, nous allons multiplier chaque trace par une fonction "gain" qui vaut 1 sur les temps d'arrivée pour lesquels les réflexions sont d'amplitude suffisante, et qui vaut une valeur supérieure à 1 pour les plus tardive (après $t_0$). Nous pouvons utiliser une fonction 'gain' linéaire d'équation
$$ g_{lin}(t) = a (t - t_0) + 1\ si\ t \geq t_0$$ 
ou exponentielle d'équation
$$ g_{exp}(t) =  a (exp^{(b*(t-t_0))} - 1) + 1\ si\ t \geq t_0$$

Dans la cellule ci-dessous, vous pouvez tester différentes fonctions gain en modifiant les paramètres. 

In [15]:
#Fonction gain
data = np.squeeze(np.asarray(dataSVD))   #data_t0
ns, ntr = np.shape(data)
a_default, b_default = 5, 0.08
tw0_default = 10
clip_default = .3
n=10 #numéro de la trace affichée ci-après
@interact
def f(a=IntSlider(value=a_default, min=1, max=20, step=1),
      b=FloatSlider(value=b_default, min=0.001, max=1, step=0.02),
      tw0=FloatSlider(value=tw0_default, min=0, max=ns*dt, step=10*dt),
      clip=FloatLogSlider(value=clip_default, min=-5, max=0, step=.5)):
    global data_g
    #data_g, fgain = user_gain(data, dx_dt, "linear", (a, b), (tw0, ns*dt), return_fgain=True)
    data_g, fgain = user_gain(data, dx_dt, "exponential", (a, b), (tw0, ns*dt), return_fgain=True)
    
    n_default = 15
    fig, ax = plt.subplots(nrows=1, ncols=3,figsize=(15, 7))
    ax[0].plot(data[:, n-1], np.arange(len(data))*dt)
    ax[0].tick_params(labelcolor='r', labelsize='large', width=3)
    ax[0].grid(True)
    ax[0].set_ylabel('Temps (ns)')
    ax[0].set_xlabel('Amplitude')
    ax[0].set_ylim(ns*dt, 0)
    ax[0].set_yticks(np.arange(0, ns*dt, step=10))
    
    ax[1].plot(fgain, np.arange(len(data_t0))*dt)
    ax[1].tick_params(labelcolor='r', labelsize='large', width=3)
    ax[1].grid(True)
    ax[1].set_ylabel('Temps (ns)')
    ax[1].set_xlabel('Amplitude')
    ax[1].set_ylim(ns*dt, 0)
    ax[1].set_yticks(np.arange(0, ns*dt, step=10))
    
    ax[2].plot(data_g[:, n-1], np.arange(len(data))*dt)
    ax[2].tick_params(labelcolor='r', labelsize='large', width=3)
    ax[2].grid(True)
    ax[2].set_ylabel('Temps (ns)')
    ax[2].set_xlabel('Amplitude')
    ax[2].set_ylim(ns*dt, 0)
    ax[2].set_yticks(np.arange(0, ns*dt, step=10))
    
    fig2, ax2 = plt.subplots(figsize=(20, 6))
    t = np.linspace(0, 1, ns) * (len(data_t0) * dt)
    x = np.linspace(0, 1, ntr) * (ntr * dx)	
    ims = ax2.imshow(data_g, extent=[np.amin(x), np.amax(x), np.amax(t), np.amin(t)], interpolation='nearest',
                     aspect='auto', cmap='seismic', 
                     vmin=-np.amax(abs(data_g)*clip), vmax=np.amax(abs(data_g)*clip))
    plt.xlabel('Distance [m]')
    plt.ylabel('Two-way travel time [ns]')
    ax2.grid('both')
    plt.minorticks_on()
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    cbar = fig2.colorbar(ims)
    cbar.ax.set_ylabel('Amplitude')

interactive(children=(IntSlider(value=5, description='a', max=20, min=1), FloatSlider(value=0.08, description=…

In [16]:
vmin=-np.amax(abs(data_g)*clip)
vmax=np.amax(abs(data_g)*clip)

In [17]:
# Sauvons les valeurs des param de gain choisis:
a = 5
b = 0.08
tw0 = 10
clip = 0.3

#### Regardons les autres radargrammes avec ces paramètres

In [18]:
num_profil_default = 1

@interact
def f(num_profil = IntSlider(value=num_profil_default, min=1, max=nombre_de_profils, step=1)):
    #Lecture du fichier de données num_profil
    data = lecture(path, num_profil)
    if (num_profil % 2) == 0:
        print("Le profil {0} est paire donc on inverse car trajet retour".format(num_profil))
        data =  np.fliplr(data)
    else:
       print("Le profil {0} est impaire donc on l'inverse pas".format(num_profil)) 
    data_t0 = time_zero(data, dx_dt, t0)  
    ns, ntr = np.shape(data_t0)
    dataSVD = filt_SVD(data_t0, ns, ntr, SVmin, SVmax) 
    data = np.squeeze(np.asarray(dataSVD))   
    data_g, fgain = user_gain(data, dx_dt, "exponential", (a, b), (tw0, ns*dt), return_fgain=True)
    
    fig, ax = plt.subplots(figsize=(20, 6))
    t = np.linspace(0, 1, ns) * (ns * dt)
    x = np.linspace(0, 1, ntr) * (ntr * dx)	
    ims = ax.imshow(data_g, extent=[np.amin(x), np.amax(x), np.amax(t), np.amin(t)], interpolation='nearest',
                    aspect='auto', cmap='seismic', vmin=vmin, vmax=vmax)
    plt.xlabel('Distance [m]')
    plt.ylabel('Two-way travel time [ns]')
    ax.grid('both')
    plt.minorticks_on()
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    cbar = fig.colorbar(ims)
    cbar.ax.set_ylabel('Amplitude')
 

interactive(children=(IntSlider(value=1, description='num_profil', max=57, min=1), Output()), _dom_classes=('w…

## Analyse de vitesse

Analysons de plus prêt une hyperbole de diffraction observée sur le radargramme de votre choix. La forme de cette hyperbole dépend la vitesse de propagation de l'onde em. Elle nous permet de faire une analyse de vitesse.

In [19]:
num_profil = 19
data = lecture(path, num_profil)
dx, dt = dx_dt
ns, ntr = data.shape
if (num_profil % 2) == 0:
    print("Le profil {0} est paire donc on inverse car trajet retour".format(num_profil))
    data =  np.fliplr(data)
else:
    print("Le profil {0} est impaire donc on l'inverse pas".format(num_profil)) 

data_t0 = time_zero(data, dx_dt, t0)  
ns, ntr = np.shape(data_t0)
dataSVD = filt_SVD(data_t0, ns, ntr, SVmin, SVmax) 
data = np.squeeze(np.asarray(dataSVD))   
data_g, fgain = user_gain(data, dx_dt, "exponential", (a, b), (tw0, ns*dt), return_fgain=True)

2023-12-07 13:44:43 - WARNING: no DZG file found for GPS input
--------------------WARNING - PLEASE READ---------------------
2023-12-07 13:44:43 - WARNING: could not read frequency for antenna name "'HS350US'"
2023-12-07 13:44:43 - WARNING: trying to use frequencies of None MHz (estimated)...
2023-12-07 13:44:43 - more info: rh_ant=['HS350US', None, None, None]
2023-12-07 13:44:43 -            known_ant=[False, None, None, None]
2023-12-07 13:44:43 - please submit a bug report with this warning, the antenna name and frequency
2023-12-07 13:44:43 - at https://github.com/iannesbitt/readgssi/issues/new
2023-12-07 13:44:43 - or send via email to ian (dot) nesbitt (at) gmail (dot) com.
2023-12-07 13:44:43 - if possible, please attach a ZIP file with the offending DZT inside.
--------------------------------------------------------------
Le profil 19 est impaire donc on l'inverse pas


In [20]:
xmax = longueur
tmax = dt*ns
x0_default = 25 # Distance (m) de l'apex de l'hyperbole le long du profil
t0_default = 50 # TWT (ns) de l'apex de l'hyperbole
v_default = 0.3 # vitesse du milieu en m/ns
r_default = 0 # rayon (m) théorique de l'objet diffractant
# Attention le calcul de l'hyperbole ne prend pas vraiment le rayon en compte. Enfin je crois...
w_default = 3 # largeur de l'hyperbole (m)µ
clip=0.3

@interact
def f(x0=FloatSlider(value=x0_default, min=0, max=xmax, step=10*dx), 
      t0=FloatSlider(value=t0_default, min=0, max=tmax, step=10*dt),
      v=FloatSlider(value=v_default, min=0.03, max=0.3, step=0.005),
      #r=FloatSlider(value=r_default, min=0, max=2, step=0.05),
      width=FloatSlider(value=w_default, min=0, max=longueur, step=0.1)
     ):
    r=0.0
    velocity_analysis(data_g, dx_dt, (x0, t0, v, r, width),clip, num_profil, False)
    print((x0, t0, v, r, width))

interactive(children=(FloatSlider(value=25.0, description='x0', max=30.0, step=0.16863406408094433), FloatSlid…

In [21]:
param =(9.27487, 10.16475, 0.095, 0.0, 0.9) #(25.63238, 56.29707, 0.3, 0.0, 11.5)
#param =(25.63238, 56.29707, 0.3, 0.0, 11.5)

## Migration de Stolt

In [22]:
data_mig, dx_dz = stolt_migration(data_g, dx_dt, param[2])

In [23]:
@interact
def f(clip=FloatLogSlider(value=clip_default, min=-5, max=0, step=.5)):
    velocity_analysis(data_mig, dx_dt, param, clip, num_profil, False)
    

interactive(children=(FloatLogSlider(value=0.3, description='clip', max=0.0, min=-5.0, step=0.5), Output()), _…

In [25]:
print('profondeur (m) =', param[1]/2 * param[2])

profondeur (m) = 0.482825625


In [28]:
eps1=3
eps2=100
ref = (np.sqrt(eps1)-np.sqrt(eps2))/(np.sqrt(eps1)+np.sqrt(eps2))

In [29]:
ref

-0.7047317922538399